In [1]:
!pip install pymilvus ollama llama-index-llms-ollama llama-index-vector-stores-milvus

  Using cached ollama-0.2.0-py3-none-any.whl.metadata (4.1 kB)
Using cached ollama-0.2.0-py3-none-any.whl (9.5 kB)


In [6]:
!pip install llama-index-embeddings-huggingface sentencepiece

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.8 MB/s eta 0:00:00m eta 0:00:01:01:01


In [2]:
from pymilvus import MilvusClient

client = MilvusClient("milvus_mistral_rag.db")

In [1]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding

embeddings = HuggingFaceEmbedding(model_name="dangvantuan/sentence-camembert-large")

/Users/stephen/Library/Caches/pypoetry/virtualenvs/fork-mistral-cookbook-9DpsfPiA-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
No sentence-transformers model found with name dangvantuan/sentence-camembert-large. Creating a new one with MEAN pooling.
/Users/stephen/Library/Caches/pypoetry/virtualenvs/fork-mistral-cookbook-9DpsfPiA-py3.11/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
from llama_index.llms.ollama import Ollama
from llama_index.vector_stores.milvus import MilvusVectorStore

from llama_index.core import StorageContext, ServiceContext

llm = Ollama(model="mistral", request_timeout=120.0)

service_context = ServiceContext.from_defaults(llm=llm, embed_model=embeddings, chunk_size=350)

vector_store = MilvusVectorStore(
    uri="milvus_mistral_rag.db",
    collection_name="mistral_french_parliament",
    dim=1024,  # the value changes with embedding model
    overwrite=True  # drop table if exist and then create
    
    )
storage_context = StorageContext.from_defaults(vector_store=vector_store)

/var/folders/kv/3dw9p_ts4b114chqt9m027pc0000gn/T/ipykernel_48132/649171650.py:8: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults(llm=llm, embed_model=embeddings, chunk_size=350)


In [6]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

docs = SimpleDirectoryReader(input_files=['data/french_parliament_discussion.xml']).load_data()
vector_index = VectorStoreIndex.from_documents(docs, storage_context=storage_context, service_context=service_context)

In [7]:
from llama_index.core.tools import RetrieverTool, ToolMetadata

milvus_tool_openai = RetrieverTool(
    retriever=vector_index.as_retriever(similarity_top_k=3),  # retrieve top_k results
    metadata=ToolMetadata(
        name="CustomRetriever",
        description='Retrieve relevant information from provided documents.'
    ),
)

In [8]:
query_engine = vector_index.as_query_engine()
response = query_engine.query("What did the French parliament talk about the last time?")

In [9]:
response.response

' The French parliament discussed a proposed law and related issues concerning the management of the ongoing health crisis. They also expressed their hope that the debates would lead to finding compromises, as indicated by the presence of various amendments. The conversation highlighted the persistence of some fundamental differences among members, which is considered normal in a democratic setting. Additionally, they reflected on the progress made over the past two years and welcomed contributions from other commissions.'